<a href="https://colab.research.google.com/github/bambutch/python_classes/blob/main/%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D1%81_%D1%81%D0%B0%D0%B9%D1%82%D0%BE%D0%B2_%D0%BD%D0%B0_Python_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_%D0%B1%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%B8_BeautifulSoup(%D0%9A%D0%BD%D0%B8%D0%B3%D0%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Парсинг данных с сайтов на Python с помощью библиотеки BeautifulSoup



## Библиотека BeautifulSoup

[Документация](http://www.crummy.com/software/BeautifulSoup/)

**Beautiful Soup** — это библиотека Python для извлечения данных из файлов HTML и XML. Она работает с вашим любимым парсером, чтобы дать вам естественные способы навигации, поиска и изменения дерева разбора. Она обычно экономит программистам часы и дни работы.


Установка библиотеки:

```python
!pip install bs4
```

BS использует парсер, самым быстрым считается **lxml**
<br>В стандартных библиотеках Python такой библиотеки , поэтому ее необходимо устанавливать
```python
!pip install lxml
```

Импортировать в код ее при этом не нужно

## Задание 3

In [ ]:
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install lxml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from bs4 import BeautifulSoup
import lxml

URL = "https://books.toscrape.com/catalogue/page-{}.html"

def get_html(page):
  """Собирает html"""
  # Инициализация сессии
  session = requests.session()
  # Указание заголовков сессии
  session.headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36",
      "Accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
  }

  try:
    # Указание заголовков сессии
    res = session.get(URL.format(page))
    # Проверка на ошибку
    res.raise_for_status()
  except Exception as ex:
    print(ex)
    return

  return res.text


def get_soup(page):
  """ Варит суп """
  # Получаем исходный код
  html = get_html(page)
  # Если нету страницы, то выходим
  if not html: return
  # Начинаем варить суп (Инициализируем bs)
  soup = BeautifulSoup(html, "lxml")
  return soup


def get_book_info(li):
  """ Собирает информацию о книге """
  # Получаем название книги, которое находится в теге <a> внутри <h3> 
  title = li.select_one("h3 a")
  if not title:
    print("Error find title!")
    return
  title = title["title"]

  # Получаем рейтинг книги, которое находится в теге <p> с классом "star-rating"
  rating = li.select_one("p.star-rating")
  if not rating:
    print("Error find rating!")
    return
  rating = rating["class"][1]

  # Словарь, который сопостовляет слово с цифрой
  WORD_TO_NUM = {
      "One": 1,
      "Two": 2,
      "Three": 3,
      "Four": 4,
      "Five": 5
  }
  rating = WORD_TO_NUM.get(rating, 0)

  # Получаем рейтинг книги, которое находится в теге <p> обернутым в тег
  # <div> с классом "product_price"
  price = li.select_one("div.product_price p")
  if not price:
    print("Error find price!")
    return
  # Получаем часть текста до знака £
  price = price.text.split("£")[1]

  # Возвращаем информацию о книге
  return {
      "title": title,
      "rating": rating,
      "price": price
  }

def get_books_info(soup):
  """ Информация о книгах на странице """
  # Получаем список книг со страницы в теге <ol> с классом "row"
  list_rows = soup.select_one("ol.row")
  # Получаем карточки о книгах из списка в теге <li>
  all_li = list_rows.select("li")
  result = []
  # Проходимся по каждой карточке и достаем из нее информацию
  for li in all_li:
    result.append(get_book_info(li))
  # Краткая запись
  # result = [get_book_info(li) for li in all_li]
  return result


In [ ]:
import time
import csv

# select - все  искомые элемент
# select_one - первый попавшийся искомый элемент

# 1. Цикл по страницам
# 2. Получить исходный код
# 3. Сварить суп из исходного кода
# 4. Получить информацию о книгах


result = []
# # Вариант с range
# # Цикл по количеству страниц
# for page in range(1, 3):
#   # Перерыв между запросами
#   time.sleep(3)
#   soup = get_soup(page)
#   if not soup:
#     # Если нету супа, то выходим из цикла => либо ошибка, либо конец страниц
#     break
#   # Получаем информацию о книгах
#   books_info = get_books_info(soup)
#   # Записываем в result получившуюся инфу о книгах на странице
#   result = result + books_info

# Вариант с циклом while
# Стартовый номер страницы
page = 1
# Бесконечный цикл
while True:
  # Перерыв между запросами
  time.sleep(3)
  soup = get_soup(page)
  if not soup:
    # Если нету супа, то выходим из цикла => либо ошибка, либо конец страниц
    break
  # Получаем информацию о книгах
  books_info = get_books_info(soup)
  # Записываем в result получившуюся инфу о книгах на странице
  result = result + books_info
  # Переходим на следующую страницу
  page = page + 1
print(f"Всего пройдено страниц: {page-1}")

with open("books.csv", "w") as f:
  writer = csv.DictWriter(f, result[0].keys())
  writer.writeheader()
  for r in result:
    writer.writerow(r)